In [3]:
import pandas as pd
import numpy as np

pd.options.display.float_format = '{:.5f}'.format

In [4]:
bureau_df = pd.read_csv('bureau.csv')
bureau_b_df = pd.read_csv('bureau_balance.csv')

In [5]:
Recent_Balance = bureau_b_df.groupby('SK_ID_BUREAU')['MONTHS_BALANCE'].first()
Transaction_Period = abs(bureau_b_df.groupby('SK_ID_BUREAU')['MONTHS_BALANCE'].last() - bureau_b_df.groupby('SK_ID_BUREAU')['MONTHS_BALANCE'].first())
Recent_Status = bureau_b_df.groupby('SK_ID_BUREAU')['STATUS'].first()
mode_status = bureau_b_df.groupby('SK_ID_BUREAU')['STATUS'].apply(lambda x : x.mode()[0])

bureau_b_pre = pd.concat([Recent_Balance, Transaction_Period, Recent_Status, mode_status], axis = 1)
bureau_b_pre.columns = ['RECENT_BALANCE', 'TRANSACTION_PERIOD', 'RECENT_STATUS', 'MODE_STATUS']
bureau_b_pre = bureau_b_pre.reset_index()
bureau_b_pre.to_csv('bureau_b_pre.csv')

In [6]:
merge_bureau = pd.merge(bureau_df, bureau_b_pre, on = 'SK_ID_BUREAU', how = 'left')

In [7]:
def object_pre (df) :
    
    pk_id = sorted(list(df['SK_ID_CURR'].unique()))
    first_id = []
    mode_id = []
    object_col_first = []
    object_col_mode = []

    for col in df:
        if df[col].dtype == 'object':
            first_id.append(col+'_FIRST')
            mode_id.append(col+'_MODE')
            print(col)
            object_col_first.append(list(df.groupby('SK_ID_CURR')[col].first()))
            object_col_mode.append(list(df.groupby('SK_ID_CURR')[col].apply(lambda x : x.mode()[0])))

    object_col_first_df = pd.DataFrame(object_col_first).transpose()
    object_col_mode_df = pd.DataFrame(object_col_mode).transpose()   

    object_col_first_df.columns = [first_id]
    object_col_mode_df.columns = [mode_id]
    return pk_id, object_col_first_df, object_col_mode_df

In [8]:
temp_merge_bureau = merge_bureau.copy()

for col in temp_merge_bureau:
    if temp_merge_bureau[col].dtype == 'object':
        temp_merge_bureau[col] = temp_merge_bureau[col].fillna('NULL')

pk_id, object_col_first, object_col_mode = object_pre(temp_merge_bureau)

merge_obj_df = pd.concat([object_col_first, object_col_mode], axis = 1)
merge_obj_df.index = pk_id
merge_obj_df = merge_obj_df.reset_index()
merge_obj_df = merge_obj_df.rename(columns={'index':'SK_ID_CURR'})
merge_obj_df = merge_obj_df.replace('NULL', np.nan)

merge_obj_df.columns = ['SK_ID_CURR', 'CREDIT_ACTIVE_FIRST', 'CREDIT_CURRENCY_FIRST', 'CREDIT_TYPE_FIRST', \
                        'RECENT_STATUS_FIRST', 'MODE_STATUS_FIRST', 'CREDIT_ACTIVE_MODE', 'CREDIT_CURRENCY_MODE', \
                        'CREDIT_TYPE_MODE', 'RECENT_STATUS_MODE', 'MODE_STATUS_MODE']

level_0 = ['SK_ID_BUREAU', 'DAYS_CREDIT', 'CREDIT_DAY_OVERDUE', 'DAYS_CREDIT_ENDDATE', 'DAYS_ENDDATE_FACT', \
    'AMT_CREDIT_MAX_OVERDUE', 'CNT_CREDIT_PROLONG', 'AMT_CREDIT_SUM', 'AMT_CREDIT_SUM_DEBT', 'AMT_CREDIT_SUM_LIMIT', \
        'AMT_CREDIT_SUM_OVERDUE', 'DAYS_CREDIT_UPDATE', 'AMT_ANNUITY', 'Recent_Balance', 'Transaction_Period']
level_1 = ['_MEAN', '_MEDIAN', '_STD', '_FIRST']

col_names = []
for name1 in level_0 :
    for name2 in level_1 :
        col_names.append(name1+name2)

merge_real_num_df = pd.DataFrame(merge_bureau.groupby('SK_ID_CURR').agg(['mean','median','std','first']))
merge_real_num_df.reset_index(drop = True, inplace = True)
merge_real_num_df.columns = merge_real_num_df.columns.droplevel(0)

merge_real_num_df.columns = col_names

CREDIT_ACTIVE
CREDIT_CURRENCY
CREDIT_TYPE
RECENT_STATUS
MODE_STATUS


In [9]:
total_bureau_df = pd.concat([merge_obj_df, merge_real_num_df], axis = 1)
total_bureau_df.to_csv('bureau_merge.csv')